In [2]:
import random
import polars as pl

import netcbs

In [2]:
# Print contexts and codebook
print(netcbs.context2types)
print(netcbs.codebook)

{'Family': {301, 302, 303, 304, 305, 306, 307, 308, 309, 310, 311, 312, 313, 314, 315, 316, 317, 318, 319, 320, 321, 322}, 'Colleagues': {201}, 'Neighbors': {101, 102}, 'Schoolmates': {501, 502, 503, 504, 505, 506}, 'Housemates': {401, 402}}
{101: 'Neighbor - 10 closest addresses', 102: 'Neighborhood acquaintance - 20 random neighbors within 200 meters', 201: 'Colleague', 301: 'Parent', 302: 'Co-parent', 303: 'Grandparent', 304: 'Child', 305: 'Grandchild', 306: 'Full sibling', 307: 'Half sibling', 308: 'Unknown sibling', 309: 'Full cousin', 310: 'Cousin', 311: 'Aunt/Uncle', 312: 'Partner - married', 313: 'Partner - not married', 314: 'Parent-in-law', 315: 'Child-in-law', 316: 'Sibling-in-law', 317: 'Stepparent', 318: 'Stepchild', 319: 'Stepsibling', 320: 'Married full cousin', 321: 'Married cousin', 322: 'Married aunt/uncle', 401: 'Housemate', 402: 'Housemate - institution', 501: 'Classmate primary education', 502: 'Classmate special education', 503: 'Classmate secondary education', 50

### Select the sample dataframe and the dataframe with the variable to aggregate

For this example we will be using synthetic data. For each context (Family, Colleagues, Neighbors, Schoolmates, Housemates), we generated a "network file" containing 1,000,000 relationships (see section below). Each relationship is taken at random from any of the context types (see netdbs.contexts2types).

We then create two files: one with the IDs (RINPERSOON) in the sample, one with the IDs (RINPERSOON) and the variable to aggregate. In the CBS RA you will use real data.

In [10]:

# Create df_sample example
df_sample = pl.DataFrame(
    {
        "RINPERSOON": range(100_000_000, 100_010_000),
        "RINPERSOONS": ["R"]*10_000
    }
    
)

df_agg = pl.LazyFrame(
    {
        "RINPERSOON":   range(100_000_000, 101_000_000),
        "RINPERSOONS":  ["R"]*1_000_000,
        "Income":       [random.normalvariate(30000, 5000) for _ in range(1_000_000)],
        "Age":          [random.normalvariate(30, 10) for _ in range(1_000_000)]
    }
)



### Run query

This is the most important part of the code. Here we will aggregate the variable of interest. In this case, we will aggregate the number of relationships per context.

In [11]:
## Query: The income of the parent's of the schoolmates of the children in the sample

## How to construct the query
# 1. Start with the variables that you want to aggregate, e.g. "[Income, Age] ->"
# 2. Then add the relationships between the tables, e.g., "[Income, Age] -> Family[301]".
# In square brackets you can specify the type of the relationships: 
# write [all] for all, or [301,302] for parents and co-parents
# 3. You can add several tables: "[Income, Age] -> Family[301] -> Schoolmates[all]"
# 4. Finally, you must write "-> Sample" 

## Other parameters
# df_sample: the sample dataframe (with the people you want to have information on)
# df_agg: the dataframe with the information you want to aggregate. For example, the income of all people in the country
# year: the year of the data you want to use
# agg_func: the aggregation function you want to use. For example, pl.mean or pl.sum
# return_pandas: if True, the function returns a pandas dataframe. If False, it returns a polars dataframe
# lazy: if True, the operations are concatenated lazily and computed at the end. If False, the operations are computed immediately
# cbdata_path: the path to the CBS data. Usually this is "G:/Bevolking". In this example, we use synthetic data saved in "cbsdata/Bevolking". 


## The transform function validates the query before running it

# Example
query =  "[Income, Age] -> Family[all] -> Schoolmates[all] -> Sample"

df = netcbs.transform(query, 
               df_sample = df_sample, 
               df_agg = df_agg, 
               year=2021,
               cbsdata_path='cbsdata/Bevolking', # Path to the CBS data ("G:/Bevolking"), in this example is synthetic data locally 
               agg_func=[pl.mean, pl.sum, pl.max], 
               return_pandas=False, 
               lazy=True)

df    

INFO:netcbs.netcbs:The dataframe may contain duplicated entries. Use a non-lazy dataframe to find if this is the case
INFO:netcbs.netcbs:The dataframe may contain duplicated entries. Use a non-lazy dataframe to find if this is the case


RINPERSOON,RINPERSOONS,mean_Income,mean_Age,sum_Income,sum_Age,max_Income,max_Age
i64,str,f64,f64,f64,f64,f64,f64
100000000,"""R""",31380.1757,28.972624,62760.351401,57.945248,34687.57934,29.336526
100000001,"""R""",32879.284118,37.345571,131517.136471,149.382285,35894.169057,44.11918
100000002,"""R""",27460.960294,30.001971,137304.80147,150.009855,37925.004174,38.651416
100000003,"""R""",29677.4618,28.657164,504516.850594,487.171792,34707.643278,44.786676
100000004,"""R""",null,null,null,null,null,null
…,…,…,…,…,…,…,…
100009995,"""R""",28769.819413,31.496589,115079.277653,125.986358,34641.216281,44.699332
100009996,"""R""",null,null,null,null,null,null
100009997,"""R""",null,null,null,null,null,null


In [7]:
pl.DataFrame(df_sample

TypeError: DataFrame constructor called with unsupported type 'LazyFrame' for the `data` parameter

In [6]:
# You can also validate the query before running it
query =  "Income -> Family[301,302,303] -> Schoolmates[all] -> Sample"
netcbs.validate_query(query, 
               df_sample = df_sample, 
               df_agg = df_agg, 
               year=2021,
               cbsdata_path='cbsdata/Bevolking', # Path to the CBS data, in this example is synthetic data locally 
)

['Sample', 'Schoolmates[all]', 'Family[301,302,303]', 'Income']

In [7]:
# Create path to latest verion of CBS data
netcbs.format_path(context='Family[301]', year=2010, cbsdata_path='cbsdata/Bevolking')

('cbsdata/Bevolking/FAMILIENETWERKTAB/FAMILIENETWERKTAB2010V1.csv', {301})

## Create synthetic data (not needed at CBS!)

Let's create some synthetic data to explain how the code works

For each context (Family, Colleagues, Neighbors, Schoolmates, Housemates), we will generate a "network file" containing 1,000,000 relationsihps. Each relationship is taken at random from any of the context types (see netdbs.contexts2types).


In [3]:
netcbs.create_synthetic_data("Family", 2021, 1_000_000, outpath="cbsdata/Bevolking")
netcbs.create_synthetic_data("Colleagues", 2021, 1_000_000, outpath="cbsdata/Bevolking")
netcbs.create_synthetic_data("Neighbors", 2021, 1_000_000, outpath="cbsdata/Bevolking")
netcbs.create_synthetic_data("Schoolmates", 2021, 1_000_000, outpath="cbsdata/Bevolking")
netcbs.create_synthetic_data("Housemates", 2021, 1_000_000, outpath="cbsdata/Bevolking")


In [19]:
# FOR TESTING (LATER ON)

# Create df_sample example
df_sample = pl.DataFrame(
    {
        "RINPERSOON": range(100_000_000, 100_000_010),
        "RINPERSOONS": ["R"]*10
    }
)

df_agg = pl.DataFrame(
    {
        "RINPERSOON":   range(100_000_000, 100_000_010),
        "RINPERSOONS":  ["R"]*10,
        "Income":       [random.normalvariate(30000, 5000) for _ in range(10)],
        "Age":          [random.normalvariate(30, 10) for _ in range(10)]
    }
)

query =  "[Income, Age] -> Family[all] -> Schoolmates[all] -> Sample"

df = netcbs.transform(query, 
               df_sample = df_sample, 
               df_agg = df_agg, 
               year=2020,
               cbsdata_path='cbsdata/Bevolking', # Path to the CBS data ("G:/Bevolking"), in this example is synthetic data locally 
               agg_func=[pl.mean, pl.sum, pl.max], 
               return_pandas=False, 
               lazy=True)

df    

RINPERSOON,RINPERSOONS,mean_Income,mean_Age,sum_Income,sum_Age,max_Income,max_Age
i64,str,f64,f64,f64,f64,f64,f64
100000000,"""R""",30441.338632,32.953794,3.5921e6,3888.547723,39104.044557,50.638837
100000001,"""R""",30494.715232,33.010881,2.6530e6,2871.946604,39104.044557,50.638837
100000002,"""R""",31312.598165,31.564705,3.4444e6,3472.117496,39104.044557,50.638837
100000003,"""R""",30861.611098,32.395738,4.5058e6,4729.777793,39104.044557,50.638837
100000004,"""R""",30862.305772,32.612148,4.1355e6,4370.027836,39104.044557,50.638837
100000005,"""R""",31541.894736,32.819419,1.7033e6,1772.24865,39104.044557,50.638837
100000006,"""R""",29841.261126,32.644702,1.9098e6,2089.260935,39104.044557,50.638837
100000007,"""R""",30759.015916,32.897355,2.2762e6,2434.404305,39104.044557,50.638837
100000008,"""R""",30710.504316,33.108053,4.2380e6,4568.911273,39104.044557,50.638837
